In [11]:
import re
import string
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import TfidfVectorizer



def retrieve_docs_and_clean():
  df = pd.read_csv('/Users/ammarlakdawala/Documents/phoenixbio/Phoenix Bioinformatics/IF Code/FinalDatasets/phyb_100.csv')
  #df = df['abstract']
  # Clean Paragraphs
  col_list =  list(df["abstract"])

  documents_clean = []
  for d in df:
      document_test = re.sub(r'[^\x00-\x7F]+', ' ', d)
      document_test = re.sub(r'@\w+', '', document_test)
      document_test = document_test.lower()
      document_test = re.sub(r'[%s]' % re.escape(string.punctuation), ' ', document_test)
      document_test = re.sub(r'[0-9]', '', document_test)
      document_test = re.sub(r'\s{2,}', ' ', document_test)
      documents_clean.append(document_test)

  return documents_clean

In [12]:
docs = retrieve_docs_and_clean()

# Create Term-Document Matrix with TF-IDF weighting
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(docs)



In [13]:
# Create a DataFrame
df = pd.DataFrame(X.T.toarray(), index=vectorizer.get_feature_names_out())
print(df.head())
print(df.shape)

            0         1    2    3    4
abstract  0.0  0.000000  0.0  1.0  0.0
journal   0.0  0.707107  0.0  0.0  0.0
label     0.0  0.000000  0.0  0.0  1.0
pubmed    0.0  0.000000  1.0  0.0  0.0
title     0.0  0.707107  0.0  0.0  0.0
(6, 5)


In [14]:
docs = retrieve_docs_and_clean()
# Create Term-Document Matrix with TF-IDF weighting
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(docs)

# Create a DataFrame
df = pd.DataFrame(X.T.toarray(), index=vectorizer.get_feature_names_out())
df.head()

,0,1,2,3,4
abstract,0.0,0.000000,0.0,1.0,0.0
journal,0.0,0.707107,0.0,0.0,0.0
label,0.0,0.000000,0.0,0.0,1.0
pubmed,0.0,0.000000,1.0,0.0,0.0
title,0.0,0.707107,0.0,0.0,0.0


In [18]:
def get_similar_articles(q, df):
  print("query:", q)
  print("The following are articles with the highest cosine similarity values: ")
  q = [q]
  q_vec = vectorizer.transform(q).toarray().reshape(df.shape[0],)
  sim = {}
for i in df.columns:
    sim[i] = np.dot(df.loc[:, i].values, q_vec) / np.linalg.norm(df.loc[:, i]) * np.linalg.norm(q_vec)
  
  sim_sorted = sorted(sim.items(), key=lambda x: x[1], reverse=True)
  
  for k, v in sim_sorted:
    if v != 0.0:
      print("Similarity Value:", v)
      print(docs[k])
      print()


q1 = 'phyB'
get_similar_articles(q1, df)
print('-'*100)